#### Search Algorithms
The aim of this notebook is to implement some search algorithms for graphs discussed in the first section of the Udacity Intro to AI course.

This will include breadth first search, uniform cost search, and A* search. 

The first goal is to implement these in a rough form. These implementations could be improved by creating a graph class, and using more efficient data structures for parts of the search algorithm.

##### Graphs
I will implement a directed graph as simply a dictionary of dictionaries. For each node $x$ in the graph we store in $g[x]$ the list of all nodes that can be reached from $x$ in one step, and the cost of this step. For breadth first search and uniform cost search are equivalent if these costs are all set to 1.

For my examples the nodes will be simply labelled with numbers.

In [1]:
graph = {}

graph[1] = {2:1, 3:2}
graph[2] = {4:1, 5:1}
graph[3] = {6:2,7:2}
graph[4] = {}
graph[5] = {6:1}
graph[6] = {}
graph[7] = {}

Note this example graph is a binary tree with 3 layers, weighted so that the paths on the right side of the tree are 2, and the paths on the left side are 1. There is also an additional connection between the two inner nodes at the bottom. 

It is easy to see that the shortest path from node 1 to node 6 is 1-3-6 without the costs, and 1-2-5-6 with the costs.

##### Breadth first search, Uniform cost search and A* search
The first two of these algorithms only differ in that uniform cost search takes account of the costs of the paths in the graph. As such we can simply implement uniform cost search, and also get breadth first search by setting all path weights to 1. 

A* differs from the other two in how it decides the cost of a path. A heuristic $h$ is provided that is an apriori estimate of the distance to the goal from each node. The algorithm then minimizes the sum of the path cost and this heuristic. As such we can acquire uniform cost search by setting the heuristic to zero.

The search algorithm operates in the following way. We initialise a frontier as the starting node, initialise a costs dictionary storing the minimum cost from the initial node to each new node discovered, and initialise a paths dictionary storing the minimal cost path from the initial node to each newly discovered node. The algorithm proceeds by popping off nodes from the frontier with the minimal cost, and exploring the neighbours of this node, updating the costs and paths dictionaries if new nodes are discovered, or if cheaper paths are found to previously discovered nodes. The algorithm ends when the frontier is empty, though it can be terminated early if the goal node is the one popped of the frontier. The minimal path and its cost are then those stored in the paths and costs dictionaries for the goal node, if they exist, else no connection exists between the initial and goal node (e.g. from a child to a parent in a directed tree, or nodes on two separate components of a disconnected graph).

I will implement the heuristic similarly to the graph. In principle we only need to define the heuristic for estimated costs to the goal node, however doing it in this way allows you to use one heuristic for finding shortest paths between multiple different nodes.

In [16]:
def search(graph,initial_node,goal_node, heuristic = None):
    
    # If not doing A*, set heuristic to 0
    if heuristic==None:
        heuristic = {}
        heuristic[goal_node] = {}
        for node in graph:
            heuristic[goal_node][node] = 0
    
    # Initialise frontier, costs and paths
    frontier = [initial_node]
    costs = {initial_node:heuristic[goal_node][initial_node]}
    paths = {initial_node:[initial_node]}
    
    # loop while frontier non-empty
    while(len(frontier)>0):
        
        # search for minimal cost frontier node
        
        # hack to deal with initialising a minimum value
        min_cost = None
        min_node = None
        for node in frontier:
            if min_cost==None:
                min_cost = costs[node]
                min_node = node
            elif costs[node] < min_cost:
                min_cost = costs[node]
                min_node = node
        
        # remove minimal cost node from frontier
        frontier.remove(min_node)
        
        # explore the minimal cost node
        for node in graph[min_node]:
            # temporary cost to node is
            # cost to get to min_node
            cost = costs[min_node]
            # plus path cost to go from min_node to node
            cost += graph[min_node][node]
            # plus the heuristic cost to the goal from node
            cost += heuristic[goal_node][node]
            
            # if node is newly discovered, initialise its cost and path
            if node not in costs:
                costs[node] = cost
                paths[node] = []
                # hack to copy a path, stored as a list of nodes
                for step in paths[min_node]:
                    paths[node].append(step)
                # add the final step
                paths[node].append(node)
                # add newly discovered nodes to the frontier
                frontier.append(node)
            # if node seen before, 
            # update its cost and path if the new path is cheaper
            elif node in costs and cost < costs[node]:
                costs[node] = cost
                paths[node] = []
                for step in paths[min_node]:
                    paths[node].append(step)
                paths[node].append(node)
            # end loop
    # check for goal
    if goal_node in costs:
        print("Cheapest path from {} to {} is:".format(initial_node, goal_node))
        print(paths[goal_node])
        print("This path has cost:")
        print(costs[goal_node])
    else:
        print("No connection found from {} to {}.".format(initial_node, goal_node))            

We can test uniform cost search on the above example graph to find the cheapest path from node 1 to node 6, expected to be 1-2-5-6.

In [17]:
search(graph,1,6)

Cheapest path from 1 to 6 is:
[1, 2, 5, 6]
This path has cost:
3


To test breadth first search we can create a new graph with all weights set to 1.

In [18]:
unit_graph = {}
# For a large graph this would be slow.
# In such a case it's probably better to modify the search algorithm directly.
for node in graph:
    unit_graph[node] = {}
    for neighbour in graph[node]:
        unit_graph[node][neighbour] = 1
search(unit_graph, 1,6)

Cheapest path from 1 to 6 is:
[1, 3, 6]
This path has cost:
2


We can use a non-admissible heuristic to demonstrate that the algorithm also does A* search. For example suppose the heuristic between nodes x and y is $h(x,y) = |x-y|^2$. This is non-admissible for the current graph, for example $h(2,6) = 16 > 2$. As such A* search with this heuristic would choose the path 1-3-6. 

In [19]:
heuristic = {}
for node in graph:
    heuristic[node] = {}
    for other_node in graph:
        heuristic[node][other_node] = (node-other_node)**2
search(graph,1,6, heuristic)

Cheapest path from 1 to 6 is:
[1, 3, 6]
This path has cost:
38


The A* search fails here because the heuristic was not admissible, it over-estimated the true distance between nodes. A simple though not very useful modification is to set the heuristic to 0.5 for all values, except of course $h(x,x) = 0$.

In [34]:
heuristic = {} 
for node in graph:
    heuristic[node] = {}
    for other_node in graph:
        if other_node != node:
            heuristic[node][other_node] = 0.5
        else:
            heuristic[node][other_node]=0
search(graph,1,6, heuristic)

Cheapest path from 1 to 6 is:
[1, 2, 5, 6]
This path has cost:
4.5


Note that in the above, setting $h(x,y) = 1$ for all $x \neq y$ seems to be admissible, but the algorithm then finds that both paths 1-3-6 and 1-2-5-6 have cost 6, and the algorithm decides on the first. This is probably either a bug in how I've written the algorithm.

##### Depth first search
This is a little different to the above algorithm, and generally less efficient. The idea is to follow each new node as it's discovere, and update costs appropriately. One benefit is it's easier to implement, as we no longer need to choose an optimal node from the frontier, simply the most recent one added to the frontier.

The code is essentially the same besides this, so below I've re-used the min_node variable name, even though it's not really minimal for anything anymore.

In [37]:
def depth_search(graph,initial_node,goal_node):
    
    # Initialise frontier, costs and paths
    frontier = [initial_node]
    costs = {initial_node:0}
    paths = {initial_node:[initial_node]}
    
    # loop while frontier non-empty
    while(len(frontier)>0):
        
        min_node = frontier.pop()
        
        # explore the minimal cost node
        for node in graph[min_node]:
            # temporary cost to node is
            # cost to get to min_node
            cost = costs[min_node]
            # plus path cost to go from min_node to node
            cost += graph[min_node][node]
            
            # if node is newly discovered, initialise its cost and path
            if node not in costs:
                costs[node] = cost
                paths[node] = []
                # hack to copy a path, stored as a list of nodes
                for step in paths[min_node]:
                    paths[node].append(step)
                # add the final step
                paths[node].append(node)
                # add newly discovered nodes to the frontier
                frontier.append(node)
            # if node seen before, 
            # update its cost and path if the new path is cheaper
            elif node in costs and cost < costs[node]:
                costs[node] = cost
                paths[node] = []
                for step in paths[min_node]:
                    paths[node].append(step)
                paths[node].append(node)
            # end loop
    # check for goal
    if goal_node in costs:
        print("Cheapest path from {} to {} is:".format(initial_node, goal_node))
        print(paths[goal_node])
        print("This path has cost:")
        print(costs[goal_node])
    else:
        print("No connection found from {} to {}.".format(initial_node, goal_node))   

In [38]:
depth_search(graph, 1,6)

Cheapest path from 1 to 6 is:
[1, 2, 5, 6]
This path has cost:
3


In [60]:
deep_tree = {}
N = 3000
for i in range(N):
    deep_tree[i] = {i+1:1}
deep_tree[N] = {}
deep_tree[0][N+1]=1
deep_tree[N+1] = {}

heuristic = {}
heuristic[N+1] = {}
for i in range(N+2):
    heuristic[N+1][i] = 0

import time

start = time.time()
depth_search(deep_tree,0,N+1)
end = time.time()
print("Depth first search took {}".format(end-start))
start = time.time()
search(deep_tree,0,N+1, heuristic)
end = time.time()
print("Breadth first search took {}".format(end-start))

Cheapest path from 0 to 3001 is:
[0, 3001]
This path has cost:
1
Depth first search took 0.5440053939819336
Cheapest path from 0 to 3001 is:
[0, 3001]
This path has cost:
1
Breadth first search took 0.5570077896118164
